In [41]:
# Imports and Setup
from collections import defaultdict
from unittest.mock import patch, mock_open

import os
import mido

In [42]:
# Mapping of MIDI drum note numbers to Clone Hero drum notes and their corresponding flags
DRUM_MAPPING = {
    35: (0, 'K'),  # Acoustic Bass Drum mapped to note 0 with flag 'K'
    36: (0, 'K'),  # Bass Drum (Kick) mapped to note 0 with flag 'K'
    38: (1, 'R'),  # Acoustic Snare mapped to note 1 with flag 'R'
    40: (1, 'R'),  # Electric Snare mapped to note 1 with flag 'R'
    42: (2, 'Y'),  # Closed Hi-Hat mapped to note 2 with flag 'Y'
    44: (2, 'Y'),  # Pedal Hi-Hat mapped to note 2 with flag 'Y'
    46: (3, 'B'),  # Open Hi-Hat mapped to note 3 with flag 'B'
    49: (4, 'G'),  # Crash Cymbal 1 mapped to note 4 with flag 'G'
    51: (4, 'G'),  # Ride Cymbal mapped to note 4 with flag 'G'
    45: (2, 'Y'),  # Low Tom mapped to note 2 with flag 'Y'
    47: (2, 'Y'),  # Mid Tom mapped to note 2 with flag 'Y'
    48: (2, 'Y'),  # High Mid Tom mapped to note 2 with flag 'Y'
    50: (4, 'G'),  # High Tom mapped to note 4 with flag 'G'
    57: (4, 'G'),  # Crash Cymbal 2 mapped to note 4 with flag 'G'
}

print(DRUM_MAPPING)


{35: (0, 'K'), 36: (0, 'K'), 38: (1, 'R'), 40: (1, 'R'), 42: (2, 'Y'), 44: (2, 'Y'), 46: (3, 'B'), 49: (4, 'G'), 51: (4, 'G'), 45: (2, 'Y'), 47: (2, 'Y'), 48: (2, 'Y'), 50: (4, 'G'), 57: (4, 'G')}


In [43]:
# File paths
midi_file_path = '../songs/midi_songs/test-split.mid'
track_name = os.path.splitext(os.path.basename(midi_file_path))[0]
chart_path = f'../songs/chart_files/{track_name}.chart'

print(midi_file_path)
print(chart_path)
print(track_name)

../songs/midi_songs/test-split.mid
../songs/chart_files/test-split.chart
test-split


In [44]:
# Create MIDI file object
if os.path.exists(midi_file_path):
    mid = mido.MidiFile(midi_file_path)
    print("Created Successfully")
else:
    print(f"File not found at {midi_file_path}")

Created Successfully


In [45]:
# Initialize song metadata
song_metadata = {
    "Name": track_name,
    "Artist": "Unknown",
    "Charter": "AI Generated",
    "Album": "Generated Charts",
    "Year": "2024",
    "Offset": 0,
    "Resolution": 192,
    "Player2": "bass",
    "Difficulty": 0,
    "PreviewStart": 0,
    "PreviewEnd": 0,
    "Genre": "rock"
}
print(song_metadata)


{'Name': 'test-split', 'Artist': 'Unknown', 'Charter': 'AI Generated', 'Album': 'Generated Charts', 'Year': '2024', 'Offset': 0, 'Resolution': 192, 'Player2': 'bass', 'Difficulty': 0, 'PreviewStart': 0, 'PreviewEnd': 0, 'Genre': 'rock'}


In [46]:
# Initialize data structure for chart
chart_data = {
    "Song": song_metadata,
    "SyncTrack": defaultdict(list),
    "Events": {},
    "ExpertDrums": defaultdict(list)
}

print(chart_data)

{'Song': {'Name': 'test-split', 'Artist': 'Unknown', 'Charter': 'AI Generated', 'Album': 'Generated Charts', 'Year': '2024', 'Offset': 0, 'Resolution': 192, 'Player2': 'bass', 'Difficulty': 0, 'PreviewStart': 0, 'PreviewEnd': 0, 'Genre': 'rock'}, 'SyncTrack': defaultdict(<class 'list'>, {}), 'Events': {}, 'ExpertDrums': defaultdict(<class 'list'>, {})}


In [47]:
# Initialize timing variables
total_ticks = 0
tempo = 120000  # Default tempo (120 BPM)
time_sig = (4, 4)  # Default time signature

In [48]:
# Initialize the variable to store the initial tempo as None
# This will hold the tempo value once it is found in the MIDI file
initial_tempo = None

# Iterate through each track in the MIDI file
for track in mid.tracks:
    # Loop through each message in the current track
    for msg in track:
        # Check if the message type is 'set_tempo', which indicates a tempo change
        if msg.type == 'set_tempo':
            # If a 'set_tempo' message is found, set initial_tempo to the tempo value from the message
            initial_tempo = msg.tempo
            # Break out of the inner loop since we have found the initial tempo
            break
    
    # If an initial tempo has been found, print it for debugging purposes
    if initial_tempo:
        print(f"Initial Tempo: {initial_tempo}")
        # Break out of the outer loop since we no longer need to search other tracks
        break


Initial Tempo: 444444


In [49]:
# Set the initial tempo to the found tempo or use a default tempo if none was found
# This ensures that there is always a valid tempo value for further processing
initial_tempo = initial_tempo or tempo

# Append the initial tempo to the SyncTrack at tick 0 in the chart data
# 'B' followed by the tempo value is the format used to denote a tempo change in chart files
chart_data["SyncTrack"][0].append(f"B {initial_tempo}")

# Append the initial time signature to the SyncTrack at tick 0 in the chart data
# 'TS' followed by the time signature numerator denotes a time signature event in chart files
chart_data["SyncTrack"][0].append(f"TS {time_sig[0]}")


print(chart_data)

{'Song': {'Name': 'test-split', 'Artist': 'Unknown', 'Charter': 'AI Generated', 'Album': 'Generated Charts', 'Year': '2024', 'Offset': 0, 'Resolution': 192, 'Player2': 'bass', 'Difficulty': 0, 'PreviewStart': 0, 'PreviewEnd': 0, 'Genre': 'rock'}, 'SyncTrack': defaultdict(<class 'list'>, {0: ['B 444444', 'TS 4']}), 'Events': {}, 'ExpertDrums': defaultdict(<class 'list'>, {})}


In [50]:
# Iterate through each track in the MIDI file
for track in mid.tracks:
    # Initialize tick counter for this specific track to accumulate time deltas
    track_ticks = 0
    
    # Process each MIDI message in the current track
    for msg in track:
        # Add the time delta of the current message to the track's tick counter
        track_ticks += msg.time
        
        # Check if the message is a tempo change
        if msg.type == 'set_tempo':
            # Update the current tempo with the new tempo value from the message
            tempo = msg.tempo
            # Print the new tempo for debugging purposes
            print(f"Tempo Change: {tempo}")
        
        # Check if the message is a time signature change
        elif msg.type == 'time_signature':
            # Update the time signature with a tuple of numerator and denominator from the message
            time_sig = (msg.numerator, msg.denominator)
            # Print the new time signature for debugging purposes
            print(f"Time Signature: {time_sig}")
    
    # Update total_ticks to be the maximum of its current value and track_ticks
    # This ensures that total_ticks reflects the longest track length found, capturing full MIDI duration
    total_ticks = max(total_ticks, track_ticks)



    print(f"Total Ticks: {total_ticks}")

Time Signature: (4, 4)
Tempo Change: 444444
Tempo Change: 1463415
Tempo Change: 458015
Tempo Change: 545455
Tempo Change: 540541
Tempo Change: 535714
Total Ticks: 88244


In [51]:
# Calculate song length in seconds by using the formula:
# (total_ticks * tempo) / (mid.ticks_per_beat * 1000000)
# - total_ticks: Total number of ticks accumulated across all tracks
# - tempo: Current tempo in microseconds per beat
# - mid.ticks_per_beat: Number of ticks per beat in the MIDI file
# - 1000000: Converts microseconds to seconds
song_length = (total_ticks * tempo) / (mid.ticks_per_beat * 1000000)

# Convert song length from seconds to milliseconds and store it in the song metadata
# Multiplying by 1000 converts seconds to milliseconds for more precise length representation
song_metadata["Length"] = int(song_length * 1000)

# Print the calculated song length in seconds for debugging purposes
print(f"Song Length: {song_length}")


Song Length: 393.94621846666666


In [52]:
# Calculate initial ticks multiplier using the formula:
# (mid.ticks_per_beat * 192) / (initial_tempo / 1000000 * 60)
# - mid.ticks_per_beat: Number of ticks per beat in the MIDI file
# - 192: A constant used for scaling, often related to the resolution or division of beats
# - initial_tempo: Initial tempo in microseconds per beat
# - 1000000: Converts microseconds to seconds
# - 60: Converts beats per second to beats per minute (BPM)
ticks_multiplier = (mid.ticks_per_beat * 192) / (initial_tempo / 1000000 * 60)

# Print the result of mid.ticks_per_beat multiplied by 192 for debugging purposes
print(mid.ticks_per_beat * 192)

# Print the result of initial_tempo divided by 1000000 and multiplied by 60 for debugging purposes
print(initial_tempo / 1000000 * 60)

# Print the calculated ticks multiplier for debugging purposes
print(f"Tick Multiplier: {ticks_multiplier}")


23040
26.66664
Tick Multiplier: 864.000864000864


In [53]:
# Initialize current_tick to zero to track the cumulative time in ticks across all messages
current_tick = 0

# Iterate through each track in the MIDI file
for track in mid.tracks:
    # Process each MIDI message in the current track
    for msg in track:
        # Accumulate the time delta of the current message into current_tick
        current_tick += msg.time
        
        # Calculate the corresponding chart tick using the ticks multiplier
        # This converts MIDI ticks to a format suitable for chart timing
        chart_tick = int(current_tick * ticks_multiplier / mid.ticks_per_beat)
        
        # Handle note messages specifically for 'note_on' events with a positive velocity
        if msg.type == 'note_on' and msg.velocity > 0:
            # Check if the message is a drum note (channel 9) and exists in the drum mapping
            if hasattr(msg, 'channel') and msg.channel == 9 and msg.note in DRUM_MAPPING:
                # Retrieve the mapped note number and flag from DRUM_MAPPING
                note_num, flag = DRUM_MAPPING[msg.note]
                
                # Format the note string for inclusion in the chart data
                note_str = f"N {note_num} 0{' ' + flag if flag else ''}"
                
                # Append the formatted note string to the ExpertDrums section at the calculated chart tick
                chart_data["ExpertDrums"][chart_tick].append(note_str)
        
        # Handle tempo change messages to adjust timing as necessary
        elif msg.type == 'set_tempo':
            # Append a tempo change event to the SyncTrack section at the calculated chart tick
            chart_data["SyncTrack"][chart_tick].append(f"B {msg.tempo}")
            
            # Recalculate the ticks multiplier based on the new tempo for accurate timing conversion
            ticks_multiplier = (mid.ticks_per_beat * 192) / (1000000 * 240 / msg.tempo)
        
        # Handle time signature messages to ensure correct musical timing
        elif msg.type == 'time_signature':
            # Append a time signature change event to the SyncTrack section at the calculated chart tick
            chart_data["SyncTrack"][chart_tick].append(f"TS {msg.numerator}")


In [54]:
# Initialize chart_text with the opening section for the song metadata
chart_text = "[Song]\n{\n"

# Iterate over each key-value pair in the song metadata dictionary
for key, value in chart_data["Song"].items():
    # Append each metadata entry to chart_text, formatting numbers and strings appropriately
    chart_text += f"  {key} = {value if isinstance(value, (int, float)) else f'{value}'}\n"

# Close the Song section with a closing brace and add two newlines for separation
chart_text += "}\n\n"

# Begin the SyncTrack section in chart_text
chart_text += "[SyncTrack]\n{\n"

# Iterate over each tick in the sorted keys of the SyncTrack dictionary
for tick in sorted(chart_data["SyncTrack"].keys()):
    # For each event at the current tick, append it to chart_text with proper formatting
    for event in chart_data["SyncTrack"][tick]:
        chart_text += f"  {tick} = {event}\n"

# Close the SyncTrack section with a closing brace and add two newlines for separation
chart_text += "}\n\n"

# Add an empty Events section to chart_text
chart_text += "[Events]\n{\n}\n\n"

# Begin the ExpertDrums section in chart_text
chart_text += "[ExpertDrums]\n{\n"

# Iterate over each tick in the sorted keys of the ExpertDrums dictionary
for tick in sorted(chart_data["ExpertDrums"].keys()):
    # For each note at the current tick, append it to chart_text with proper formatting
    for note in chart_data["ExpertDrums"][tick]:
        chart_text += f"  {tick} = {note}\n"

# Close the ExpertDrums section with a closing brace
chart_text += "}\n"

# Print the final constructed chart text for debugging purposes
print(chart_text)


[Song]
{
  Name = test-split
  Artist = Unknown
  Charter = AI Generated
  Album = Generated Charts
  Year = 2024
  Offset = 0
  Resolution = 192
  Player2 = bass
  Difficulty = 0
  PreviewStart = 0
  PreviewEnd = 0
  Genre = rock
  Length = 393946
}

[SyncTrack]
{
  0 = B 444444
  0 = TS 4
  0 = TS 4
  0 = B 444444
  30548 = B 1463415
  32323 = B 545455
  38159 = B 535714
  38504 = B 540541
  101996 = B 458015
}

[Events]
{
}

[ExpertDrums]
{
  1365 = N 3 0 B
  1450 = N 0 0 K
  1450 = N 3 0 B
  1535 = N 3 0 B
  1621 = N 0 0 K
  1621 = N 3 0 B
  1706 = N 3 0 B
  1791 = N 0 0 K
  1791 = N 3 0 B
  1877 = N 3 0 B
  1962 = N 0 0 K
  1962 = N 3 0 B
  2047 = N 3 0 B
  2133 = N 0 0 K
  2133 = N 3 0 B
  2218 = N 3 0 B
  2303 = N 0 0 K
  2303 = N 3 0 B
  2389 = N 3 0 B
  2474 = N 0 0 K
  2474 = N 3 0 B
  2559 = N 3 0 B
  2645 = N 0 0 K
  2645 = N 3 0 B
  2730 = N 3 0 B
  2815 = N 3 0 B
  2815 = N 0 0 K
  2901 = N 3 0 B
  2986 = N 3 0 B
  2986 = N 0 0 K
  3071 = N 3 0 B
  3157 = N 3 0 B
  3157 =

In [55]:
# Create the output directory for the chart file if it does not already exist
# os.makedirs ensures that all intermediate directories are created as needed
# The 'exist_ok=True' parameter prevents an error if the directory already exists
os.makedirs(os.path.dirname(chart_path), exist_ok=True)

# Open the specified chart file path in write mode ('w') to create or overwrite the file
with open(chart_path, 'w') as f:
    # Write the constructed chart text to the file
    f.write(chart_text)

# Verify that the chart file was successfully created and written
if os.path.exists(os.path.dirname(chart_path)) and os.path.isfile(chart_path):
    # Print a success message indicating the chart file's creation path
    print(f"Chart file successfully created at: {chart_path}")
    
    # Further verify that the chart file is not empty by checking its size
    if os.path.getsize(chart_path) > 0:
        # Print a confirmation message that the chart file contains data
        print("Chart file contains data")
    else:
        # Print a warning message if the chart file is found to be empty
        print("Warning: Chart file is empty")
else:
    # Print an error message if the chart file was not successfully created
    print("Error: Failed to create chart file")


Chart file successfully created at: ../songs/chart_files/test-split.chart
Chart file contains data
